# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f92ec699b20>
├── 'a' --> tensor([[-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717]])
└── 'x' --> <FastTreeValue 0x7f92ec699a00>
    └── 'c' --> tensor([[ 0.2235,  2.0399, -1.6537,  0.1490],
                        [ 1.7428, -0.5446, -2.2239,  1.1660],
                        [ 1.3051,  0.7493,  1.9194,  1.2377]])

In [4]:
t.a

tensor([[-0.1524, -2.3242,  0.4819],
        [-0.0212,  0.6493, -0.4717]])

In [5]:
%timeit t.a

71.3 ns ± 0.972 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f92ec699b20>
├── 'a' --> tensor([[ 0.3013,  1.3969, -1.0710],
│                   [ 0.6843, -1.1032,  2.7671]])
└── 'x' --> <FastTreeValue 0x7f92ec699a00>
    └── 'c' --> tensor([[ 0.2235,  2.0399, -1.6537,  0.1490],
                        [ 1.7428, -0.5446, -2.2239,  1.1660],
                        [ 1.3051,  0.7493,  1.9194,  1.2377]])

In [7]:
%timeit t.a = new_value

72.7 ns ± 0.634 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717]]),
    x: Batch(
           c: tensor([[ 0.2235,  2.0399, -1.6537,  0.1490],
                      [ 1.7428, -0.5446, -2.2239,  1.1660],
                      [ 1.3051,  0.7493,  1.9194,  1.2377]]),
       ),
)

In [10]:
b.a

tensor([[-0.1524, -2.3242,  0.4819],
        [-0.0212,  0.6493, -0.4717]])

In [11]:
%timeit b.a

65.1 ns ± 0.771 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0267,  0.9639,  0.1608],
               [-1.6692,  0.6706,  1.3130]]),
    x: Batch(
           c: tensor([[ 0.2235,  2.0399, -1.6537,  0.1490],
                      [ 1.7428, -0.5446, -2.2239,  1.1660],
                      [ 1.3051,  0.7493,  1.9194,  1.2377]]),
       ),
)

In [13]:
%timeit b.a = new_value

615 ns ± 9.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1e+03 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.3 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

290 µs ± 5.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

276 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9235926e50>
├── 'a' --> tensor([[[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]],
│           
│                   [[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]],
│           
│                   [[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]],
│           
│                   [[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]],
│           
│                   [[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]],
│           
│                   [[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]],
│           
│                   [[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]],
│           
│                   [[-0.1524, -2.3242,  0.4819],
│                    [-0.0212,  0.6493, -0.4717]]])
└── 'x' --> <FastTreeValue 0x7f9235926700>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.3 µs ± 992 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f92ec699310>
├── 'a' --> tensor([[-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717],
│                   [-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717],
│                   [-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717],
│                   [-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717],
│                   [-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717],
│                   [-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717],
│                   [-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717],
│                   [-0.1524, -2.3242,  0.4819],
│                   [-0.0212,  0.6493, -0.4717]])
└── 'x' --> <FastTreeValue 0x7f92ec6992e0>
    └── 'c' --> tensor([[ 0.2235,  2.0399, -1.6537,  0.1490],
                        [ 1.7428, -0.5446, -2.2239,  1.1660],
                 

In [23]:
%timeit t_cat(trees)

41.2 µs ± 407 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82.5 µs ± 1.77 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]],
       
               [[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]],
       
               [[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]],
       
               [[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]],
       
               [[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]],
       
               [[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]],
       
               [[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]],
       
               [[-0.1524, -2.3242,  0.4819],
                [-0.0212,  0.6493, -0.4717]]]),
    x: Batch(
           c: tensor([[[ 0.2235,  2.0399, -1.6537,  0.1490],
                       [ 1.7428, -0.5446, -2.2239,  1.1660],
                       [ 1.3051,  0.7493,  1.9194,  1.2377]],
         

In [26]:
%timeit Batch.stack(batches)

105 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717],
               [-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717],
               [-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717],
               [-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717],
               [-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717],
               [-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717],
               [-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717],
               [-0.1524, -2.3242,  0.4819],
               [-0.0212,  0.6493, -0.4717]]),
    x: Batch(
           c: tensor([[ 0.2235,  2.0399, -1.6537,  0.1490],
                      [ 1.7428, -0.5446, -2.2239,  1.1660],
                      [ 1.3051,  0.7493,  1.9194,  1.2377],
                      [ 0.2235,  2.0399, -1.6537,  0.1490],
                      [ 1.7428, -

In [28]:
%timeit Batch.cat(batches)

184 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

589 µs ± 5.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
